In [ ]:
import gc

import re

import sys

import time

import jieba

import os.path

import os

import datetime

import numpy as np

import pandas as pd

import lightgbm as lgb

import gensim 

from gensim.models import Word2Vec  

In [ ]:
######################################## data read #####################################

#工作空间设置

data_path = '/Users/hubei/Downloads/BDcloud下载/Database Castle药物分子/晶泰科技比赛数据/'

os.chdir(data_path)#设置当前工作空间

print (os.getcwd())#获得当前工作目录



#数据读取

df_protein_train    = pd.read_csv('df_protein_train.csv')#1653

df_protein_test     = pd.read_csv('df_protein_test.csv')#414

protein_concat = pd.concat([df_protein_train,df_protein_test])

# print(protein_concat.tail(10))

df_molecule         = pd.read_csv('df_molecule.csv')#111216

df_affinity_train   = pd.read_csv('df_affinity_train.csv')#165084

df_affinity_test    = pd.read_csv('df_affinity_test_toBePredicted.csv')#41383

df_affinity_test['Ki'] = -11

data  =  pd.concat([df_affinity_train,df_affinity_test])

data.tail()

In [ ]:
data.head()

In [ ]:
###########                                 feature                               ############
###############################################################################################

#1、Fingerprint分子指纹处理展开

feat = []

for i in range(0,len(df_molecule)):

    feat.append(df_molecule['Fingerprint'][i].split(','))

feat = pd.DataFrame(feat)

# print(feat.head())

feat = feat.astype('int')

feat.columns=["Fingerprint_{0}".format(i) for i in range(0,167)]

# print(feat.head())

feat["Molecule_ID"] = df_molecule['Molecule_ID']

# print(feat.head())

data = data.merge(feat, on='Molecule_ID', how='left')

data.tail()

In [ ]:
data.head()

In [ ]:
feat.head()

In [ ]:
#2、df_molecule其他特征处理

feat = df_molecule.drop('Fingerprint',axis=1)          #删除Fingerprint列

# print(feat.head())

data = data.merge(feat, on='Molecule_ID', how='left')
data.head()

In [ ]:
#3、protein 蛋白质 词向量训练 看不懂的加公众号：Python_R_wu，稍后会讲解下，来不及写

n = 128

texts = [[word for word in re.findall(r'.{3}',document)]  
         # re.findall:在字符串中找到正则表达式所匹配的所有子串，并返回一个列表，如果没有找到匹配的，则返回空列表。

               for document in list(protein_concat['Sequence'])]

# print(texts)

model = Word2Vec(texts,size=n,window=4,min_count=1,negative=3,

                 sg=1,sample=0.001,hs=1,workers=4)          # sg ---> skip-gram模型
model.wv.vocab

In [ ]:
vectors = pd.DataFrame([model[word] for word in (model.wv.vocab)])

print(vectors.head(6))

vectors['Word'] = list(model.wv.vocab)

vectors.columns= ["vec_{0}".format(i) for i in range(0,n)]+["Word"]
vectors.head()

In [ ]:
wide_vec = pd.DataFrame()

result1=[]

aa = list(protein_concat['Protein_ID'])
# print(texts) 
# texts是蛋白质序列 3 分词之后的数组

for i in range(len(texts)):

    result2=[]         

    for w in range(len(texts[i])):

        result2.append(aa[i])    

    result1.extend(result2)
    
wide_vec['Id'] = result1

wide_vec.tail() 

In [ ]:
result1=[]

for i in range(len(texts)):

    result2=[]         

    for w in range(len(texts[i])):

        result2.append(texts[i][w])    

    result1.extend(result2)

wide_vec['Word'] = result1

wide_vec.head() 

In [ ]:
del result1,result2


wide_vec = wide_vec.merge(vectors,on='Word', how='left')

print(wide_vec.head())

wide_vec = wide_vec.drop('Word',axis=1)

wide_vec.columns = ['Protein_ID']+["vec_{0}".format(i) for i in range(0,n)]

wide_vec.head()

In [ ]:
del vectors



name = ["vec_{0}".format(i) for i in range(0,n)]



feat = pd.DataFrame(wide_vec.groupby(['Protein_ID'])[name].agg('mean')).reset_index()
# 按蛋白质ID分组后将每个ID按平均值聚集
print(feat.head())

feat.columns=["Protein_ID"]+["mean_ci_{0}".format(i) for i in range(0,n)]

data = data.merge(feat, on='Protein_ID', how='left')
data.head()

# data.tail()

In [ ]:
'''#################################### lgb ############################

train_feat = data[data['Ki']> -11].fillna(0)

testt_feat = data[data['Ki']<=-11].fillna(0)

label_x  = train_feat['Ki']

label_y  = testt_feat['Ki']



submission = testt_feat[['Protein_ID','Molecule_ID']]

len(testt_feat)

train_feat = train_feat.drop('Ki',axis=1)

testt_feat = testt_feat.drop('Ki',axis=1)

train_feat = train_feat.drop('Protein_ID',axis=1)

testt_feat = testt_feat.drop('Protein_ID',axis=1)

train_feat = train_feat.drop('Molecule_ID',axis=1)

testt_feat = testt_feat.drop('Molecule_ID',axis=1)
'''

In [ ]:
'''#lgb算法

train = lgb.Dataset(train_feat, label=label_x)

test  = lgb.Dataset(testt_feat, label=label_y,reference=train)
'''

In [ ]:
'''params = {

    'boosting_type': 'gbdt',

    'objective': 'regression_l2',

    'metric': 'l2',

    #'objective': 'multiclass',

    #'metric': 'multi_error',

    #'num_class':5,

    'min_child_weight': 3,

    'num_leaves': 2 ** 5,

    'lambda_l2': 10,

    'subsample': 0.7,

    'colsample_bytree': 0.7,

    'colsample_bylevel': 0.7,

    'learning_rate': 0.05,

    'tree_method': 'exact',

    'seed': 2017,

    'nthread': 12,

    'silent': True

    }
'''

In [ ]:
'''num_round = 1000

gbm = lgb.train(params, 

                  train, 

                  num_round, 

                  verbose_eval=50,

                  valid_sets=[train,test]

                  )
 


preds_sub = gbm.predict(testt_feat)
'''

In [ ]:
"""################ xgb ####################

import xgboost as xgb

from sklearn.cross_validation import train_test_split

import time 

start_time = time.time()



# 读入数据

train_feat = data[data['Ki']> -11].fillna(0)

testt_feat = data[data['Ki']<=-11].fillna(0)

label_x  = train_feat['Ki']

label_y  = testt_feat['Ki']

submission = testt_feat[['Protein_ID','Molecule_ID']]

train_feat = train_feat.drop('Ki',axis=1)

testt_feat = testt_feat.drop('Ki',axis=1)

train_feat = train_feat.drop('Protein_ID',axis=1)

testt_feat = testt_feat.drop('Protein_ID',axis=1)

train_feat = train_feat.drop('Molecule_ID',axis=1)

testt_feat = testt_feat.drop('Molecule_ID',axis=1)


xgb_train = xgb.DMatrix(train_feat,label = label_x)

xgb_test = xgb.DMatrix(testt_feat,label = label_y)
"""

In [ ]:
"""
params = { 

    'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    
    'max_depth':6, # 构建树的深度，越大越容易过拟合
    
    'lambda':0.5,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    
    'subsample':0.8, # 随机采样训练样本
    
    'colsample_bytree':0.7, # 生成树时进行的列采样
    
    'min_child_weight':3, 
    
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
    
    'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
    
    'eta': 0.07, # 如同学习率
    
    'seed':2018,
    
    #'nthread':0,# cpu 线程数
    
    #'eval_metric': 'auc'

}

#plst = list(params.items())

num_round = 10000 # 迭代次数
evallist=[(xgb_test,'test'), (xgb_train, 'train')]

model = xgb.train(params=params, dtrain=xgb_train, num_boost_round=num_round, evals=evallist, verbose_eval=50,) 

preds_sub = model.predict(xgb.DMatrix(testt_feat, label=label_y))

"""

In [ ]:
"""
#结果保存

nowTime=datetime.datetime.now().strftime('%m%d%H%M')#现在

# name='lgb_'+nowTime+'.csv'

name='xgb_'+nowTime+'.csv'

submission['Ki'] = preds_sub

submission.to_csv(name, index=False)

#输出运行时长

cost_time = time.time()-start_time

print ("xgboost success!",'\n',"cost time:",cost_time,"(s)......")
"""

In [ ]:
############### 逻辑回归 ##################

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
train_feat = data[data['Ki']> -11].fillna(0)

testt_feat = data[data['Ki']<=-11].fillna(0)

label_x  = train_feat['Ki']

label_y  = testt_feat['Ki']

submission = testt_feat[['Protein_ID','Molecule_ID']]

train_feat = train_feat.drop('Ki',axis=1)

testt_feat = testt_feat.drop('Ki',axis=1)

train_feat = train_feat.drop('Protein_ID',axis=1)

testt_feat = testt_feat.drop('Protein_ID',axis=1)

train_feat = train_feat.drop('Molecule_ID',axis=1)

testt_feat = testt_feat.drop('Molecule_ID',axis=1)


In [ ]:
le = preprocessing.LabelEncoder()
encoded = le.fit_transform(label_x)
cls = LogisticRegression()
cls.fit(X = train_feat,y = encoded)